In [1]:
# !git clone https://github.com/lamisgosu11/VoiceMe

In [2]:
# !cd /content/VoiceMe
# !main_dir=$PWD

In [3]:
# !preprocessing_env="$main_dir/preprocessing-env"
# # !conda create --prefix $preprocessing_env python=3.7
# # !conda activate $preprocessing_env
# !pip install Cython
# !pip install git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[all]
# !pip install requests

In [4]:
# !pip install virtualenv
# !python3 -m virtualenv $preprocessing_env
# !source $preprocessing_env/bin/activate


### inference of VITS

In [1]:
%rm -rf /content/vits

In [2]:
!git clone https://github.com/jaywalnut310/vits/

Cloning into 'vits'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81
Receiving objects: 100% (81/81), 3.33 MiB | 19.07 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [3]:
!sudo apt-get install espeak-ng
!pip install unidecode
!pip install phonemizer

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
espeak-ng is already the newest version (1.50+dfsg-10).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [4]:
%mkdir /content/vits/monotonic_align/monotonic_align
%cd /content/vits/monotonic_align
!python setup.py build_ext --inplace

/content/vits/monotonic_align
Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/usr/local/lib/python3.10/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /content/vits/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
performance hint: core.pyx:7:5: Exception check on 'maximum_path_each' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_each' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
	2. Use an 'int' return type on 'maximum_path_each' to allow an error code to be returned.
performance hint: core.pyx:38:6: Exception check on 'maximum_path_c' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_c' as 'noexcept' if you control the definition and you're sure you don't want the func

In [5]:
%cd /content/vits

/content/vits


In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

LJ Speaker

In [7]:
!gdown 'https://drive.google.com/uc?id=11aHOlhnxzjpdWDpsz1vFDCzbeEfoIxru' #vctk
!gdown 'https://drive.google.com/uc?id=1q86w74Ygw2hNzYP9cWkeClGT5X25PvBT' #lj speech

Downloading...
From (original): https://drive.google.com/uc?id=11aHOlhnxzjpdWDpsz1vFDCzbeEfoIxru
From (redirected): https://drive.google.com/uc?id=11aHOlhnxzjpdWDpsz1vFDCzbeEfoIxru&confirm=t&uuid=15e10c54-8102-4bd8-8897-9afb488a0595
To: /content/vits/pretrained_vctk.pth
100% 159M/159M [00:02<00:00, 70.3MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1q86w74Ygw2hNzYP9cWkeClGT5X25PvBT
From (redirected): https://drive.google.com/uc?id=1q86w74Ygw2hNzYP9cWkeClGT5X25PvBT&confirm=t&uuid=d7e90830-fb94-4302-8d98-f41f8b369cee
To: /content/vits/pretrained_ljs.pth
100% 146M/146M [00:02<00:00, 69.6MB/s]


In [8]:
hps = utils.get_hparams_from_file("./configs/ljs_base.json")

In [9]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("pretrained_ljs.pth", net_g, None)

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [10]:
stn_tst = get_text("I'm shocked too I'm supposed to be locked up too", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))
#save audio

In [11]:
write('nwordinparis_lj_speech.wav', hps.data.sampling_rate, audio)

vctk

In [12]:
hps = utils.get_hparams_from_file("./configs/vctk_base.json")

In [14]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("pretrained_vctk.pth", net_g, None)

In [15]:
stn_tst = get_text("I'm shocked too I'm supposed to be locked up too", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    sid = torch.LongTensor([4]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:797: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv_transpose1d(


In [16]:
write('nwordinparis_vctk.wav', hps.data.sampling_rate, audio)

### training of VITS

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [26]:
!gdown 'https://drive.google.com/uc?id=1scuFwqh8s7KIYAfZW1Eu6088ZAK2SI-v'
# esd dataset training
#rename to dataset.zip

Downloading...
From (original): https://drive.google.com/uc?id=1scuFwqh8s7KIYAfZW1Eu6088ZAK2SI-v
From (redirected): https://drive.google.com/uc?id=1scuFwqh8s7KIYAfZW1Eu6088ZAK2SI-v&confirm=t&uuid=4e543c84-f678-4d01-a085-064f1a15fde5
To: /content/vits/Emotional Speech Dataset (ESD).zip
100% 2.45G/2.45G [00:43<00:00, 55.7MB/s]


In [29]:
#unzip it to dataset folder
!unzip -q /content/vits/dataset.zip -d /content/vits/dataset

In [30]:
!python train_ms.py -c configs/vctk_base.json -m vctk_base

2024-06-10 09:33:42.180298: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 09:33:42.180356: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 09:33:42.297111: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-10 09:33:42.555852: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 09:33:44.487318: W tensorflow/compiler/tf2